# Analiz

In [82]:
import pandas as pd
df = pd.read_csv("sales.csv")
df

,order_id,date,item_name,item_type,item_price,quantity,transaction_amount,transaction_type,received_by,time_of_sale
0,1,07-03-2022,Aalopuri,Fastfood,20,13,260,NaN,Mr.,Night
1,2,8/23/2022,Vadapav,Fastfood,20,15,300,Cash,Mr.,Afternoon
2,3,11/20/2022,Vadapav,Fastfood,20,1,20,Cash,Mr.,Afternoon
3,4,02-03-2023,Sugarcane juice,Beverages,25,6,150,Online,Mr.,Night
4,5,10-02-2022,Sugarcane juice,Beverages,25,8,200,Online,Mr.,Evening
...,...,...,...,...,...,...,...,...,...,...
995,996,3/19/2023,Frankie,Fastfood,50,10,500,NaN,Mrs.,Evening
996,997,9/20/2022,Sandwich,Fastfood,60,7,420,NaN,Mr.,Evening
997,998,1/26/2023,Sandwich,Fastfood,60,13,780,NaN,Mr.,Evening
998,999,8/27/2022,Panipuri,Fastfood,20,5,100,NaN,Mrs.,Afternoon


In [83]:
df.columns

Index(['order_id', 'date', 'item_name', 'item_type', 'item_price', 'quantity',
       'transaction_amount', 'transaction_type', 'received_by',
       'time_of_sale'],
      dtype='object')

Veri kümesi 1000 satır ve 10 sütun içermektedir. 
Veri kümesinin her satırı bir siparişe ilişkin bilgileri içerir.

Amacımız, diğer sütunlardaki değerleri kullanarak "ücretler" sütunundaki değeri tahmin etmenin bir yolunu bulmaktır. 
Bunu geçmiş veriler için yapabilirsek, o zaman sadece yaş, cinsiyet, BMI, no gibi bilgileri sorarak yeni müşterilerin ücretlerini de tahmin edebilmeliyiz. 
çocukların sigara içme alışkanlıkları ve bölgeleri.

Her sütun için veri türünü kontrol edelim:

In [84]:
df.describe()

,order_id,item_price,quantity,transaction_amount
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,33.315000,8.162000,275.230000
std,288.819436,14.921744,4.413075,204.402979
min,1.000000,20.000000,1.000000,20.000000
25%,250.750000,20.000000,4.000000,120.000000
50%,500.500000,25.000000,8.000000,240.000000
75%,750.250000,50.000000,12.000000,360.000000
max,1000.000000,60.000000,15.000000,900.000000


In [85]:
df.item_name.unique()

array(['Aalopuri', 'Vadapav', 'Sugarcane juice', 'Panipuri', 'Frankie',
       'Sandwich', 'Cold coffee'], dtype=object)

In [86]:
df.query("item_type == 'Fastfood'").item_name.unique()

array(['Aalopuri', 'Vadapav', 'Panipuri', 'Frankie', 'Sandwich'],
      dtype=object)

In [87]:
df.query("item_type == 'Beverages'").item_name.unique()

array(['Sugarcane juice', 'Cold coffee'], dtype=object)

# Ön İşleme

In [88]:
# Tarih sütunundan yıl, ay ve gün bilgilerini çıkararak yeni sütunlar ekleme
df["date"] = df["date"].str.replace("/", "-")
date = pd.to_datetime(df["date"])
df['year'] = date.dt.year
df['month'] = date.dt.month
df['day'] = date.dt.day

In [106]:
from sklearn import preprocessing

# Kategorik verileri one-hot encoding ile dönüştürme
enc = preprocessing.OneHotEncoder()
categorical_columns = ['item_name', 'item_type', 'transaction_type', 'received_by', 'time_of_sale']
enc.fit(df[categorical_columns])
enc.categories_

[array(['Aalopuri', 'Cold coffee', 'Frankie', 'Panipuri', 'Sandwich',
        'Sugarcane juice', 'Vadapav'], dtype=object),
 array(['Beverages', 'Fastfood'], dtype=object),
 array(['Cash', 'Online', nan], dtype=object),
 array(['Mr.', 'Mrs.'], dtype=object),
 array(['Afternoon', 'Evening', 'Midnight', 'Morning', 'Night'],
       dtype=object)]

In [109]:
one_hot = enc.transform(df[categorical_columns]).toarray()
one_hot

array([[1., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 1., 0., 0.]])

In [111]:
# Hedef değişken ve açıklayıcı değişkenlerin ayrılması
X = sales_data_encoded.drop(['quantity', 'date', 'order_id'], axis=1)
y = sales_data_encoded['quantity']

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli seçme ve eğitme
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Test verisi üzerinde tahmin yapma
y_pred = model.predict(X_test)

# Model performansını değerlendirme
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Root Mean Squared Error: {rmse}')

Root Mean Squared Error: 0.07288689868556628


c:\Users\ismai\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [92]:
df

,order_id,date,item_name,item_type,item_price,quantity,transaction_amount,transaction_type,received_by,time_of_sale,year,month,day
0,1,07-03-2022,Aalopuri,Fastfood,20,13,260,NaN,Mr.,Night,2022,7,3
1,2,8-23-2022,Vadapav,Fastfood,20,15,300,Cash,Mr.,Afternoon,2022,8,23
2,3,11-20-2022,Vadapav,Fastfood,20,1,20,Cash,Mr.,Afternoon,2022,11,20
3,4,02-03-2023,Sugarcane juice,Beverages,25,6,150,Online,Mr.,Night,2023,2,3
4,5,10-02-2022,Sugarcane juice,Beverages,25,8,200,Online,Mr.,Evening,2022,10,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,3-19-2023,Frankie,Fastfood,50,10,500,NaN,Mrs.,Evening,2023,3,19
996,997,9-20-2022,Sandwich,Fastfood,60,7,420,NaN,Mr.,Evening,2022,9,20
997,998,1-26-2023,Sandwich,Fastfood,60,13,780,NaN,Mr.,Evening,2023,1,26
998,999,8-27-2022,Panipuri,Fastfood,20,5,100,NaN,Mrs.,Afternoon,2022,8,27
